In [ ]:
from vpython import *


QE = 1.602176565e-19        # C, electron charge
EPS_0 = 8.85418782e-12      # C/V/m, vacuum perm.
ME = 9.10938215e-31         # kg, electron mass

# converts physical position x to a logical coordinate l
def XtoL(x, dx, x0=0): 
    return (x-x0)/dx

ni = 21     # number of nodes
x0 = 0      # origin
xd = 0.1    # opposite end
x = 0
v = 0
dx = (xd-x0)/(ni-1)     # node spacing

rho = [QE*1e12]*ni
ef = [0]*ni
phi = [0]*ni

# generate a test electron
m = ME
q = -QE
x = 4*dx    # four cells from left edge
v = 0       # stationary

dt = 1e-10  # timestep

def main():
    global ef, phi, rho, x, v, phi_max
    # solve potential
    solvePotentialGS(dx,phi,rho,5000)
    #solvePotentialDirect(dx,phi,rho)  # alternate solver

    # compute electric field
    ef = computeEF(dx, ef, phi, True)

    # velocity rewind
    li = XtoL(x,dx)
    ef_p = gather(li,ef)
    v -= 0.5*(q/m)*ef_p*dt

    # save initial potential for PE calculation
    phi_max = phi[0]
    for i in range(1,ni):
        if (phi[i]>phi_max): phi_max = phi[i]

    print("time,x,v,KE,PE")
    x_old = x

    strs = []
    outstrs = []
    oscillation = graph(title="particle oscillation", xtitle='time', ytitle='x position', fast=False, width=800)
    funct1 = gcurve(color=color.blue, width=4, markers=True, marker_color=color.orange, label='particle motion')


    # particle loop
    for ts in range(1,4001):
        # sample mesh data at particle position
        li = XtoL(x,dx)
        ef_p = gather(li,ef)

        # integrate velocity and position
        x_old = x
        v += (q/m)*ef_p*dt
        x += v*dt

        phi_p = gather(XtoL(0.5*(x+x_old),dx),phi)
        ke = 0.5*m*v*v/QE
        pe = q*(phi_p-phi_max)/QE

        outstrs.append("{}, {}, {}, {}".format(ts*dt, x, v, ke, pe))

        if (ts==1 or ts%25==0):
            rate(100)
            funct1.plot(ts*dt,x)

        if (ts==1 or ts%1000==0):  # screen output every 1000 timesteps
            strs.append("ts: {}, x:{}, v:{}, KE:{}, PE:{}".format(ts,x,v,ke,pe))

    for s in strs:
        print(s)

    print("\n\n\n\n")
    s_n = "\n".join(outstrs)
    #print(s_n)
    outstrs = []
    s_n = ""

    # ouput to a CSV file for plotting
    outputCSV(x0,dx,phi,rho,ef);	


# plots the CSV data
def outputCSV(x0, dx, phi, rho, ef):

    g1 = graph(fast=False)

    funct1 = gdots(color=color.red, size=6, label='phi')
    funct2 = gdots(color=color.green, label='rho')
    funct3 = gcurve(color=color.blue, label='elec field')

    for i in range(len(phi)):
        funct1.plot(x0+i*dx, phi[i])
        funct2.plot(x0+i*dx, rho[i])
        funct3.plot(x0+i*dx, ef[i])


# solves Poisson's equation with Dirichlet boundaries using the Thomas algorithm
def solvePotentialDirect(dx, phi, rho):
    ni = len(phi)       # number of mesh nodes
    a = [0]*ni          # allocate memory for matrix coefficients
    b = [0]*ni
    c = [0]*ni
    d = [0]*ni

    # set coefficients
    for i in range(ni):
        if (i==0) or (i==ni-1):
            b[i] = 1
            d[i] = 0
        else:
            a[i] = 1/(dx*dx)
            b[i] = -2/(dx*dx)
            c[i] = 1/(dx*dx)
            d[i] = -rho[i]/EPS_0
    # initialize
    c[0] = c[0]/b[0]
    d[0] = d[0]/b[0]
    # forward step
    for i in range(1,ni):
        if (i<ni-1):
            c[i] = c[i]/(b[i]-a[i]*c[i-1])
        d[i] = (d[i] - a[i]*d[i-1])/(b[i] - a[i]*c[i-1])
    # backward substitution
    phi[ni-1] = d[ni-1]
    for i in range(ni-2,0,-1):
        phi[i] = d[i] - c[i]*phi[i+1]


# solves potential using the Gauss Seidel Method, returns true if converged
def solvePotentialGS(dx, phi, rho, max_it = 5000):
    L2 = None
    dx2 = dx*dx;		# precompute dx*dx
    w = 1.4
    ni = len(phi)
    
    # solve potential
    for solver_it in range(max_it):
        phi[0] = 0          # dirichlet boundary on left
        phi[ni-1] = 0       # dirichlet boundary on right
        
        # Gauss Seidel method, phi[i-1]-2*phi[i]+phi[i+1] = -dx^2*rho[i]/eps_0
        for i in range(1, ni-1):
            g = 0.5*(phi[i-1] + phi[i+1] + dx2*rho[i]/EPS_0)
            phi[i] = phi[i] + w*(g-phi[i])  # SOR
        
        # check for convergence
        if (solver_it%50==0):
            sum = 0
            
            # internal nodes, automatically satisfied on Dirichlet boundaries
            for i in range(1, ni-1):
                R = -rho[i]/EPS_0 - (phi[i-1] - 2*phi[i] + phi[i+1])/dx2
                sum+=R*R
            L2 = sqrt(sum)/ni
            if (L2<1e-6):
                print("Gauss-Seidel converged after {} iterations".format(solver_it))
                return False
    print("Gauss-Seidel failed to converge, L2=",L2)
    return True
    
def computeEF(dx, ef, phi, second_order=True):
    ni = len(phi)
    
    # central difference on internal nodes
    for i in range(1, ni-1):
        ef[i] = -(phi[i+1]-phi[i-1])/(2*dx)
    # boundaries
    if (second_order):
        ef[0] = (3*phi[0]-4*phi[1]+phi[2])/(2*dx)
        ef[ni-1] = (-phi[ni-3]+4*phi[ni-2]-3*phi[ni-1])/(2*dx)
    else:
        ef[0] = (phi[0]-phi[1])/dx
        ef[ni-1] = (phi[ni-2]-phi[ni-1])/dx
    return ef

def gather(li, f):
    i = int(li)
    di = li-i
    return f[i]*(1-di) + f[i+1]*(di)

main()

scene = canvas()
side = 4.0
thk = 0.3
s2 = 2*side - thk
s3 = 2*side + thk

wallR = box (pos=vector( side, 0, 0), size=vector(thk, s2, s3),  color = color.red)
wallL = box (pos=vector(-side, 0, 0), size=vector(thk, s2, s3),  color = color.red)

ball = sphere (color = color.green, radius = 0.4)
ball.mass = 1.0
ball.p = vector (-0.15, -0.23, +0.27)

# VPython particle motion simulation
while True:
    # sample mesh data at particle position
    li = XtoL(x,dx)
    ef_p = gather(li,ef)
    
    # integrate velocity and position
    x_old = x
    v += (q/m)*ef_p*dt
    x += v*dt
    
    phi_p = gather(XtoL(0.5*(x+x_old),dx),phi)
    ke = 0.5*m*v*v/QE
    pe = q*(phi_p-phi_max)/QE
    
    rate(400)
    ball.pos.x = x*2*side/xd - side



<IPython.core.display.Javascript object>

Gauss-Seidel converged after 400 iterations
time,x,v,KE,PE


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

ts: 1, x:0.020000477391142878, v:4773.91142877102, KE:6.478840157052801e-05, PE:8.142688544524454
ts: 1000, x:0.025967930717647532, v:-1016866.364133255, KE:2.9395192930241474, PE:5.239943273856877
ts: 2000, x:0.041497309733909526, v:-1624386.2828259654, KE:7.501138387953782, PE:0.6905287377496022
ts: 3000, x:0.060409586511715976, v:-1585624.5485511478, KE:7.147419501798294, PE:1.015332919350172
ts: 4000, x:0.07518028388288808, v:-916002.9842749893, KE:2.385296264794243, PE:5.767230340040832







<IPython.core.display.Javascript object>